# LABS-9: Analytics Project

In this notebook you will run and edit the code to perform some data cleaning and run a basic kNN model.

**Data**\
This dataset comes from IMDB and can be accessed on [Kaggle](https://www.kaggle.com/datasets/ashpalsingh1525/imdb-movies-dataset).

## Set up environment

In [2]:
## import packages
# ADD what each package is

import pandas as pd #data ingestion & cleaning
import numpy as np #numbers

# modeling 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [3]:
# Read in data
data = pd.read_csv("imdb_movies.csv")

In [4]:
data

,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country
0,Creed III,03/02/2023,73.0,"Drama, Action","After dominating the boxing world, Adonis Cree...","Michael B. Jordan, Adonis Creed, Tessa Thompso...",Creed III,Released,English,75000000.0,2.716167e+08,AU
1,Avatar: The Way of Water,12/15/2022,78.0,"Science Fiction, Adventure, Action",Set more than a decade after the events of the...,"Sam Worthington, Jake Sully, Zoe Saldaña, Neyt...",Avatar: The Way of Water,Released,English,460000000.0,2.316795e+09,AU
2,The Super Mario Bros. Movie,04/05/2023,76.0,"Animation, Adventure, Family, Fantasy, Comedy","While working underground to fix a water main,...","Chris Pratt, Mario (voice), Anya Taylor-Joy, P...",The Super Mario Bros. Movie,Released,English,100000000.0,7.244590e+08,AU
3,Mummies,01/05/2023,70.0,"Animation, Comedy, Family, Adventure, Fantasy","Through a series of unfortunate events, three ...","Óscar Barberán, Thut (voice), Ana Esther Albor...",Momias,Released,"Spanish, Castilian",12300000.0,3.420000e+07,AU
4,Supercell,03/17/2023,61.0,Action,Good-hearted teenager William always lived in ...,"Skeet Ulrich, Roy Cameron, Anne Heche, Dr Quin...",Supercell,Released,English,77000000.0,3.409420e+08,US
...,...,...,...,...,...,...,...,...,...,...,...,...
10173,20th Century Women,12/28/2016,73.0,Drama,"In 1979 Santa Barbara, California, Dorothea Fi...","Annette Bening, Dorothea Fields, Lucas Jade Zu...",20th Century Women,Released,English,7000000.0,9.353729e+06,US
10174,Delta Force 2: The Colombian Connection,08/24/1990,54.0,Action,When DEA agents are taken captive by a ruthles...,"Chuck Norris, Col. Scott McCoy, Billy Drago, R...",Delta Force 2: The Colombian Connection,Released,English,9145817.8,6.698361e+06,US
10175,The Russia House,12/21/1990,61.0,"Drama, Thriller, Romance","Barley Scott Blair, a Lisbon-based editor of R...","Sean Connery, Bartholomew 'Barley' Scott Blair...",The Russia House,Released,English,21800000.0,2.299799e+07,US
10176,Darkman II: The Return of Durant,07/11/1995,55.0,"Action, Adventure, Science Fiction, Thriller, ...",Darkman and Durant return and they hate each o...,"Larry Drake, Robert G. Durant, Arnold Vosloo, ...",Darkman II: The Return of Durant,Released,English,116000000.0,4.756613e+08,US


## Data Cleaning & Model Prep

Before building a machine learning model, it is essential to clean and format the data. Raw data often contains missing values, inconsistent formats, or irrelevant information that can negatively impact or break a model. 
Many algorithms, including kNN, require numeric input or specificly formatted categorical data. By cleaning the data (removing or imputing missing values, converting strings to categorical variables, and creating dummy variables), we ensure that our dataset is structured in a way that the model can interpret and learn from effectively. 

Proper data preparation leads to more accurate, reliable, and interpretable results.

There are many decisions that get made throughout this process and there is often no "right" answer - so documentating why you do things as you clean data is **key**.

### Missing Values

We saw in our design lab that some of our columns are missing values. Many models can not tolerate missing data (they will break the model), so we have to deal with these before passing the data through to our model.

We can use the [`.info()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html#pandas.DataFrame.info) method to see what columns are missing data. Run this below (look back at LABS-06 if you don't remember how). Put your code below.

In [5]:
#YOUR CODE HERE


2 columns are missing data: `genre` and `crew`. 

Since we have a large data set for kNN, we can drop the relatively few rows that are missing data using .dropna()

In [6]:
## make a new df to make changes to
model_data = data.copy()

In [7]:
model_data.dropna(inplace=True)

In [8]:
model_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10052 entries, 0 to 10177
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   names       10052 non-null  object 
 1   date_x      10052 non-null  object 
 2   score       10052 non-null  float64
 3   genre       10052 non-null  object 
 4   overview    10052 non-null  object 
 5   crew        10052 non-null  object 
 6   orig_title  10052 non-null  object 
 7   status      10052 non-null  object 
 8   orig_lang   10052 non-null  object 
 9   budget_x    10052 non-null  float64
 10  revenue     10052 non-null  float64
 11  country     10052 non-null  object 
dtypes: float64(3), object(9)
memory usage: 1020.9+ KB


### Look at some columns

In this step, we will look at some of the columns to see what values they contain. This will help us decide how to format them for our model. The status column is a good one to start with.

In [9]:
data['status'].value_counts()

status
Released           10131
Post Production       31
In Production         16
Name: count, dtype: int64

As you can tell from the code, most of the movies are "Released," with a few in "Post Production" or "In Production." Let's look at countries next.

In [10]:
data['country'].value_counts()

country
AU    4885
US    2750
JP     538
KR     361
FR     222
GB     174
ES     153
HK     125
IT     123
MX     105
CN      93
DE      88
CA      67
RU      52
IN      43
PH      43
AR      41
BR      38
TH      30
DK      24
PL      22
TR      20
NO      16
NL      16
CO      14
TW      13
ID      12
IE      11
CL       9
SE       9
BE       7
PE       7
FI       6
GR       6
CH       5
SU       5
UA       4
SG       4
VN       3
HU       3
ZA       3
IR       2
PR       2
CZ       2
GT       2
IS       2
SK       2
UY       2
AT       2
MY       2
LV       1
KH       1
PT       1
XC       1
IL       1
MU       1
PY       1
DO       1
BO       1
BY       1
Name: count, dtype: int64

As you can see, there are a lot of different countries, some with only one or two movies. Let's collapse these countries into "Other" for our model.

#### Collapse columns with too many levels

In [11]:
model_data['country'] = model_data['country'].apply(lambda x: x if model_data['country'].value_counts()[x] > 100 else 'other')

Use .value_counts() to see the levels of the genre column. Write your code below.

In [12]:
#YOUR CODE HERE

Let's look at the other columns to see if we need to collapse any others.

In [13]:
for col in ['genre', 'orig_lang']:
    print(f"Value counts for {col}:")
    print(model_data[col].value_counts())
    print("\n")

Value counts for genre:
genre
Drama                                                   556
Comedy                                                  373
Drama, Romance                                          268
Horror                                                  258
Horror, Thriller                                        202
                                                       ... 
Action, Animation, Crime, Drama                           1
Adventure, Animation, Family, Action                      1
Drama, Animation, Family, Comedy, Fantasy                 1
Science Fiction, War                                      1
Action, Adventure, Science Fiction, Thriller, Horror      1
Name: count, Length: 2300, dtype: int64


Value counts for orig_lang:
orig_lang
English                                7381
Japanese                                675
Spanish, Castilian                      388
Korean                                  384
French                                  282
Chinese   

From this outcome, we can see that genre and language also have many levels with few occurrences. We will collapse these as well by getting the top values.

#### Grab the top values for some 

This will allow you to see the top genres and languages in the dataset.

In [14]:
def get_top_value(old_column_name, new_column_name):
    """
    Function to extract the first value from a column that contains multiple comma seperated values
    Appends a new column to the dataframe
    """

    col = list(model_data[old_column_name].values)

    top_list = []
    for item in col:
        item = str(item).split(",")
        item1 = item[0]
        top_list.append(item1)

    model_data[new_column_name] = top_list 

In [15]:
get_top_value('genre', 'top_genre')
get_top_value('orig_lang', 'top_lang')

In [16]:
# look at value counts again
for col in ['top_genre', 'top_lang']:
    print(f"Value counts for {col}:")
    print(model_data[col].value_counts())
    print("\n")

Value counts for top_genre:
top_genre
Drama              1865
Action             1563
Comedy             1377
Horror              931
Animation           885
Thriller            577
Adventure           571
Romance             413
Crime               371
Family              333
Science Fiction     313
Fantasy             263
Documentary         176
Mystery             108
War                  77
Music                76
Western              72
History              46
TV Movie             35
Name: count, dtype: int64


Value counts for top_lang:
top_lang
English           7381
Japanese           675
Spanish            388
Korean             384
French             282
Chinese            144
Italian            142
Cantonese          141
German              89
Russian             65
Tagalog             42
Portuguese          35
Thai                33
Norwegian           29
Hindi               26
Polish              26
Danish              23
Swedish             22
Turkish             21
Dutch

In [17]:
#collapse top_lang
model_data['top_lang'] = model_data['top_lang'].apply(lambda x: x if model_data['top_lang'].value_counts()[x] > 10 else 'other')

Let's take a look at the top languages now.

In [37]:
model_data['top_lang'].value_counts()

top_lang
 English       7381
 Japanese       675
 Spanish        388
 Korean         384
 French         282
 Chinese        144
 Italian        142
 Cantonese      141
 German          89
other            72
 Russian         65
 Tagalog         42
 Portuguese      35
 Thai            33
 Norwegian       29
 Hindi           26
 Polish          26
 Danish          23
 Swedish         22
 Turkish         21
 Dutch           21
 Indonesian      11
Name: count, dtype: int64

#### Date

Now we will clean the date column to extract the year. This will allow us to use the year as a numeric variable in our model instead of the entire date string.

In [19]:
model_data['date_x'] = pd.to_datetime(model_data['date_x'])

In [20]:
model_data['year'] = model_data['date_x'].dt.year

#### Score

*Narrative about predicting score as high/low - avoid saying it is the target*

In [21]:
## reformat score
model_data['score'] = model_data['score'].apply(lambda x: 'high' if model_data['score'].value_counts()[x] > 70 else 'low')
model_data

,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country,top_genre,top_lang,year
0,Creed III,2023-03-02,high,"Drama, Action","After dominating the boxing world, Adonis Cree...","Michael B. Jordan, Adonis Creed, Tessa Thompso...",Creed III,Released,English,75000000.0,2.716167e+08,AU,Drama,English,2023
1,Avatar: The Way of Water,2022-12-15,high,"Science Fiction, Adventure, Action",Set more than a decade after the events of the...,"Sam Worthington, Jake Sully, Zoe Saldaña, Neyt...",Avatar: The Way of Water,Released,English,460000000.0,2.316795e+09,AU,Science Fiction,English,2022
2,The Super Mario Bros. Movie,2023-04-05,high,"Animation, Adventure, Family, Fantasy, Comedy","While working underground to fix a water main,...","Chris Pratt, Mario (voice), Anya Taylor-Joy, P...",The Super Mario Bros. Movie,Released,English,100000000.0,7.244590e+08,AU,Animation,English,2023
3,Mummies,2023-01-05,high,"Animation, Comedy, Family, Adventure, Fantasy","Through a series of unfortunate events, three ...","Óscar Barberán, Thut (voice), Ana Esther Albor...",Momias,Released,"Spanish, Castilian",12300000.0,3.420000e+07,AU,Animation,Spanish,2023
4,Supercell,2023-03-17,high,Action,Good-hearted teenager William always lived in ...,"Skeet Ulrich, Roy Cameron, Anne Heche, Dr Quin...",Supercell,Released,English,77000000.0,3.409420e+08,US,Action,English,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10173,20th Century Women,2016-12-28,high,Drama,"In 1979 Santa Barbara, California, Dorothea Fi...","Annette Bening, Dorothea Fields, Lucas Jade Zu...",20th Century Women,Released,English,7000000.0,9.353729e+06,US,Drama,English,2016
10174,Delta Force 2: The Colombian Connection,1990-08-24,high,Action,When DEA agents are taken captive by a ruthles...,"Chuck Norris, Col. Scott McCoy, Billy Drago, R...",Delta Force 2: The Colombian Connection,Released,English,9145817.8,6.698361e+06,US,Action,English,1990
10175,The Russia House,1990-12-21,high,"Drama, Thriller, Romance","Barley Scott Blair, a Lisbon-based editor of R...","Sean Connery, Bartholomew 'Barley' Scott Blair...",The Russia House,Released,English,21800000.0,2.299799e+07,US,Drama,English,1990
10176,Darkman II: The Return of Durant,1995-07-11,high,"Action, Adventure, Science Fiction, Thriller, ...",Darkman and Durant return and they hate each o...,"Larry Drake, Robert G. Durant, Arnold Vosloo, ...",Darkman II: The Return of Durant,Released,English,116000000.0,4.756613e+08,US,Action,English,1995


In [22]:
model_data['score'] = model_data['score'].astype('category')

In [23]:
model_data.dtypes

names                 object
date_x        datetime64[ns]
score               category
genre                 object
overview              object
crew                  object
orig_title            object
status                object
orig_lang             object
budget_x             float64
revenue              float64
country               object
top_genre             object
top_lang              object
year                   int32
dtype: object

### drop the columns you won't use

*narrative about which columns we can use and which we can not*

In [24]:
model_data = model_data.drop(columns=['date_x', 'names', 'genre', 'overview', 'crew', 'orig_title', 'orig_lang'])
model_data.head()

,score,status,budget_x,revenue,country,top_genre,top_lang,year
0,high,Released,75000000.0,2.716167e+08,AU,Drama,English,2023
1,high,Released,460000000.0,2.316795e+09,AU,Science Fiction,English,2022
2,high,Released,100000000.0,7.244590e+08,AU,Animation,English,2023
3,high,Released,12300000.0,3.420000e+07,AU,Animation,Spanish,2023
4,high,Released,77000000.0,3.409420e+08,US,Action,English,2023


### Train/test split

In [25]:
# features: all columns except 'score'
features = model_data.drop('score', axis=1)
# Target: score column
target = model_data['score']

In [26]:
# make columns 

#### Dummy variables

*narrative about why we need dummy vars - connect to distance + formats referenced above*

In [27]:
features = pd.get_dummies(features)

#### Split data

*narrative about why we need to split data, what train does & what test does*

In [28]:
# train test split
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=45)


## make model

*add narrative about knn object and drop down with attributes*

In [29]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(features_train, target_train)

KNeighborsClassifier()

In [30]:
target_predicted = knn.predict(features_test)

In [31]:
print("Accuracy:", accuracy_score(target_test, target_predicted))

Accuracy: 0.9408254599701641
